<a href="https://colab.research.google.com/github/KoreTane/interesting/blob/main/eng_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("MyApplication").getOrCreate()
products = spark.read.csv(["products.csv"], inferSchema=True, header=True)

In [3]:
# Definição da segmentação por Necessidade do Consumidor
segmentacoes_necessidade = { "beleza_saude": "Higiene Pessoal", "informatica_acessorios": "Tecnologia e Acessórios", "automotivo": "Automóveis e Ferramentas", "cama_mesa_banho": "Casa e Decoração", "moveis_decoracao": "Casa e Decoração", "esporte_lazer": "Esporte e Lazer", "perfumaria": "Higiene Pessoal", "utilidades_domesticas": "Casa e Decoração", "telefonia": "Tecnologia e Acessórios", "relogios_presentes": "Moda e Acessórios", "alimentos_bebidas": "Alimentação e Bebidas", "bebes": "Infantil", "papelaria": "Trabalho e Estudo", "tablets_impressao_imagem": "Tecnologia e Acessórios", "brinquedos": "Infantil", "telefonia_fixa": "Tecnologia e Acessórios", "ferramentas_jardim": "Automóveis e Ferramentas", "fashion_bolsas_e_acessorios": "Moda e Acessórios", "eletroportateis": "Casa e Decoração", "consoles_games": "Entretenimento", "audio": "Entretenimento", "fashion_calcados": "Moda e Acessórios", "cool_stuff": "Entretenimento", "malas_acessorios": "Moda e Acessórios", "climatizacao": "Casa e Decoração", "construcao_ferramentas_construcao": "Automóveis e Ferramentas", "moveis_cozinha_area_de_servico_jantar_e_jardim": "Casa e Decoração", "construcao_ferramentas_jardim": "Automóveis e Ferramentas", "fashion_roupa_masculina": "Moda e Acessórios", "pet_shop": "Animais de Estimação", "moveis_escritorio": "Casa e Decoração", "market_place": "Serviços e Outros", "eletronicos": "Tecnologia e Acessórios", "eletrodomesticos": "Casa e Decoração", "artigos_de_festas": "Entretenimento", "casa_conforto": "Casa e Decoração", "construcao_ferramentas_ferramentas": "Automóveis e Ferramentas", "agro_industria_e_comercio": "Automóveis e Ferramentas", "moveis_colchao_e_estofado": "Casa e Decoração", "livros_tecnicos": "Trabalho e Estudo", "casa_construcao": "Automóveis e Ferramentas", "instrumentos_musicais": "Entretenimento", "moveis_sala": "Casa e Decoração", "construcao_ferramentas_iluminacao": "Automóveis e Ferramentas", "industria_comercio_e_negocios": "Trabalho e Estudo", "alimentos": "Alimentação e Bebidas", "artes": "Entretenimento", "moveis_quarto": "Casa e Decoração", "livros_interesse_geral": "Entretenimento", "construcao_ferramentas_seguranca": "Automóveis e Ferramentas", "fashion_underwear_e_moda_praia": "Moda e Acessórios", "fashion_esporte": "Moda e Acessórios", "sinalizacao_e_seguranca": "Automóveis e Ferramentas", "pcs": "Tecnologia e Acessórios", "artigos_de_natal": "Entretenimento", "fashion_roupa_feminina": "Moda e Acessórios", "eletrodomesticos_2": "Casa e Decoração", "livros_importados": "Entretenimento", "bebidas": "Alimentação e Bebidas", "cine_foto": "Entretenimento", "la_cuisine": "Casa e Decoração", "musica": "Entretenimento", "casa_conforto_2": "Casa e Decoração", "portateis_casa_forno_e_cafe": "Casa e Decoração", "cds_dvds_musicais": "Entretenimento", "dvds_blu_ray": "Entretenimento", "flores": "Moda e Acessórios", "artes_e_artesanato": "Entretenimento", "fraldas_higiene": "Infantil", "fashion_roupa_infanto_juvenil": "Infantil", "seguros_e_servicos": "Serviços e Outros" }

# Criação de uma função UDF para mapear as segmentações
def segmentacoe_necessidade(categoria):
    return segmentacoes_necessidade.get(categoria, "Necessidades")

segmentacoe_necessidade_udf = udf(segmentacoe_necessidade, StringType())

# Adição da coluna de segmentação de sentimento ao DataFrame
products = products.withColumn(
    "segmentacao_necessidade",
    segmentacoe_necessidade_udf(col("product_category_name"))
)

In [4]:
# Definição da segmentação de sentimento
segmentacoes = {
    "beleza_saude": "Satisfação Alta",
    "informatica_acessorios": "Misto",
    "automotivo": "Satisfação Alta",
    "cama_mesa_banho": "Neutro",
    "moveis_decoracao": "Satisfação Alta",
    "esporte_lazer": "Satisfação Alta",
    "perfumaria": "Satisfação Alta",
    "utilidades_domesticas": "Neutro",
    "telefonia": "Misto",
    "relogios_presentes": "Satisfação Alta",
    "alimentos_bebidas": "Satisfação Alta",
    "bebes": "Satisfação Alta",
    "papelaria": "Neutro",
    "tablets_impressao_imagem": "Misto",
    "brinquedos": "Satisfação Alta",
    "telefonia_fixa": "Neutro",
    "ferramentas_jardim": "Satisfação Alta",
    "fashion_bolsas_e_acessorios": "Satisfação Alta",
    "eletroportateis": "Neutro",
    "consoles_games": "Satisfação Alta",
    "audio": "Satisfação Alta",
    "fashion_calcados": "Misto",
    "cool_stuff": "Satisfação Alta",
    "malas_acessorios": "Satisfação Alta",
    "climatizacao": "Neutro",
    "construcao_ferramentas_construcao": "Satisfação Alta",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Satisfação Alta",
    "construcao_ferramentas_jardim": "Satisfação Alta",
    "fashion_roupa_masculina": "Misto",
    "pet_shop": "Satisfação Alta",
    "moveis_escritorio": "Satisfação Alta",
    "market_place": "Neutro",
    "eletronicos": "Misto",
    "eletrodomesticos": "Neutro",
    "artigos_de_festas": "Satisfação Alta",
    "casa_conforto": "Satisfação Alta",
    "construcao_ferramentas_ferramentas": "Satisfação Alta",
    "agro_industria_e_comercio": "Satisfação Alta",
    "moveis_colchao_e_estofado": "Satisfação Alta",
    "livros_tecnicos": "Misto",
    "casa_construcao": "Satisfação Alta",
    "instrumentos_musicais": "Satisfação Alta",
    "moveis_sala": "Satisfação Alta",
    "construcao_ferramentas_iluminacao": "Satisfação Alta",
    "industria_comercio_e_negocios": "Satisfação Alta",
    "alimentos": "Satisfação Alta",
    "artes": "Satisfação Alta",
    "moveis_quarto": "Satisfação Alta",
    "livros_interesse_geral": "Misto",
    "construcao_ferramentas_seguranca": "Satisfação Alta",
    "fashion_underwear_e_moda_praia": "Satisfação Alta",
    "fashion_esporte": "Satisfação Alta",
    "sinalizacao_e_seguranca": "Neutro",
    "pcs": "Neutro",
    "artigos_de_natal": "Satisfação Alta",
    "fashion_roupa_feminina": "Misto",
    "eletrodomesticos_2": "Neutro",
    "livros_importados": "Misto",
    "bebidas": "Satisfação Alta",
    "cine_foto": "Satisfação Alta",
    "la_cuisine": "Satisfação Alta",
    "musica": "Satisfação Alta",
    "casa_conforto_2": "Satisfação Alta",
    "portateis_casa_forno_e_cafe": "Satisfação Alta",
    "cds_dvds_musicais": "Satisfação Alta",
    "dvds_blu_ray": "Satisfação Alta",
    "flores": "Satisfação Alta",
    "artes_e_artesanato": "Satisfação Alta",
    "fraldas_higiene": "Satisfação Alta",
    "fashion_roupa_infanto_juvenil": "Satisfação Alta",
    "seguros_e_servicos": "Neutro"
}

# Criação de uma função UDF para mapear as segmentações
def segmentacao_sentimento(categoria):
    return segmentacoes.get(categoria, "Neutro")

segmentacao_udf = udf(segmentacao_sentimento, StringType())

# Adição da coluna de segmentação de sentimento ao DataFrame
products = products.withColumn(
    "segmentacao_sentimento",
    segmentacao_udf(col("product_category_name"))
)


In [5]:
# Definição da segmentação por tipo
# Definição da segmentação por tipo
segmentacoes_tipo = { "beleza_saude": "Uso Pessoal",
                     "informatica_acessorios": "Reposição",
                     "automotivo": "Reposição",
                     "cama_mesa_banho": "Uso Pessoal",
                     "moveis_decoracao": "Decorativo",
                     "esporte_lazer": "Esportivo",
                     "perfumaria": "Uso Pessoal",
                     "utilidades_domesticas": "Reposição",
                     "telefonia": "Reposição",
                     "relogios_presentes": "Presente",
                     "alimentos_bebidas": "Reposição",
                     "bebes": "Uso Pessoal",
                     "papelaria": "Educacional",
                     "tablets_impressao_imagem": "Reposição",
                     "brinquedos": "Presente",
                     "telefonia_fixa": "Reposição",
                     "ferramentas_jardim": "Reposição",
                     "fashion_bolsas_e_acessorios": "Presente",
                     "eletroportateis": "Reposição",
                     "consoles_games": "Presente",
                     "audio": "Uso Pessoal",
                     "fashion_calcados": "Uso Pessoal",
                     "cool_stuff": "Presente",
                     "malas_acessorios": "Presente",
                     "climatizacao": "Reposição",
                     "construcao_ferramentas_construcao": "Reposição",
                     "moveis_cozinha_area_de_servico_jantar_e_jardim": "Decorativo",
                     "construcao_ferramentas_jardim": "Reposição",
                     "fashion_roupa_masculina": "Uso Pessoal",
                     "pet_shop": "Reposição",
                     "moveis_escritorio": "Reposição",
                     "market_place": "Uso Pessoal",
                     "eletronicos": "Reposição",
                     "eletrodomesticos": "Reposição",
                     "artigos_de_festas": "Presente",
                     "casa_conforto": "Decorativo",
                     "construcao_ferramentas_ferramentas": "Reposição",
                     "agro_industria_e_comercio": "Reposição",
                     "moveis_colchao_e_estofado": "Decorativo",
                     "livros_tecnicos": "Educacional",
                     "casa_construcao": "Reposição",
                     "instrumentos_musicais": "Uso Pessoal",
                     "moveis_sala": "Decorativo",
                     "construcao_ferramentas_iluminacao": "Reposição",
                     "industria_comercio_e_negocios": "Reposição",
                     "alimentos": "Reposição",
                     "artes": "Uso Pessoal",
                     "moveis_quarto": "Decorativo",
                     "livros_interesse_geral": "Educacional",
                     "construcao_ferramentas_seguranca": "Reposição",
                     "fashion_underwear_e_moda_praia": "Uso Pessoal",
                     "fashion_esporte": "Uso Pessoal",
                     "sinalizacao_e_seguranca": "Reposição",
                     "pcs": "Reposição",
                     "artigos_de_natal": "Presente",
                     "fashion_roupa_feminina": "Uso Pessoal",
                     "eletrodomesticos_2": "Reposição",
                     "livros_importados": "Educacional",
                     "bebidas": "Reposição",
                     "cine_foto": "Uso Pessoal",
                     "la_cuisine": "Decorativo",
                     "musica": "Uso Pessoal",
                     "casa_conforto_2": "Decorativo",
                     "portateis_casa_forno_e_cafe": "Reposição",
                     "cds_dvds_musicais": "Uso Pessoal",
                     "dvds_blu_ray": "Uso Pessoal",
                     "flores": "Presente",
                     "artes_e_artesanato": "Uso Pessoal",
                     "fraldas_higiene": "Reposição",
                     "fashion_roupa_infanto_juvenil": "Uso Pessoal",
                     "seguros_e_servicos": "Uso Pessoal" }

# Criação de uma função UDF para mapear as segmentações de tipo
def segmentacao_tipo(categoria):
    return segmentacoes_tipo.get(categoria, "Uso Pessoal")

segmentacao_tipo_udf = udf(segmentacao_tipo, StringType())

# Adição da coluna de segmentação de tipo ao DataFrame
products = products.withColumn(
    "segmentacao_tipo",
    segmentacao_tipo_udf(col("product_category_name"))
)

In [6]:
# Definição da segmentação por validade
segmentacoes_validade = {
    "beleza_saude": "Média Duração",
    "informatica_acessorios": "Longa Duração",
    "automotivo": "Longa Duração",
    "cama_mesa_banho": "Média Duração",
    "moveis_decoracao": "Longa Duração",
    "esporte_lazer": "Média Duração",
    "perfumaria": "Longa Duração",
    "utilidades_domesticas": "Longa Duração",
    "telefonia": "Longa Duração",
    "relogios_presentes": "Longa Duração",
    "alimentos_bebidas": "Curta Duração",
    "bebes": "Curta Duração",
    "papelaria": "Média Duração",
    "tablets_impressao_imagem": "Longa Duração",
    "brinquedos": "Média Duração",
    "telefonia_fixa": "Longa Duração",
    "ferramentas_jardim": "Longa Duração",
    "fashion_bolsas_e_acessorios": "Longa Duração",
    "eletroportateis": "Média Duração",
    "consoles_games": "Longa Duração",
    "audio": "Longa Duração",
    "fashion_calcados": "Média Duração",
    "cool_stuff": "Longa Duração",
    "malas_acessorios": "Longa Duração",
    "climatizacao": "Longa Duração",
    "construcao_ferramentas_construcao": "Longa Duração",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Longa Duração",
    "construcao_ferramentas_jardim": "Longa Duração",
    "fashion_roupa_masculina": "Média Duração",
    "pet_shop": "Média Duração",
    "moveis_escritorio": "Longa Duração",
    "market_place": "Não Perecíveis",
    "eletronicos": "Longa Duração",
    "eletrodomesticos": "Longa Duração",
    "artigos_de_festas": "Média Duração",
    "casa_conforto": "Longa Duração",
    "construcao_ferramentas_ferramentas": "Longa Duração",
    "agro_industria_e_comercio": "Longa Duração",
    "moveis_colchao_e_estofado": "Longa Duração",
    "livros_tecnicos": "Não Perecíveis",
    "casa_construcao": "Longa Duração",
    "instrumentos_musicais": "Não Perecíveis",
    "moveis_sala": "Longa Duração",
    "construcao_ferramentas_iluminacao": "Longa Duração",
    "industria_comercio_e_negocios": "Longa Duração",
    "alimentos": "Curta Duração",
    "artes": "Não Perecíveis",
    "moveis_quarto": "Longa Duração",
    "livros_interesse_geral": "Não Perecíveis",
    "construcao_ferramentas_seguranca": "Longa Duração",
    "fashion_underwear_e_moda_praia": "Média Duração",
    "fashion_esporte": "Média Duração",
    "sinalizacao_e_seguranca": "Longa Duração",
    "pcs": "Longa Duração",
    "artigos_de_natal": "Média Duração",
    "fashion_roupa_feminina": "Média Duração",
    "eletrodomesticos_2": "Longa Duração",
    "livros_importados": "Não Perecíveis",
    "bebidas": "Curta Duração",
    "cine_foto": "Longa Duração",
    "la_cuisine": "Longa Duração",
    "musica": "Longa Duração",
    "casa_conforto_2": "Longa Duração",
    "portateis_casa_forno_e_cafe": "Longa Duração",
    "cds_dvds_musicais": "Longa Duração",
    "dvds_blu_ray": "Longa Duração",
    "flores": "Curta Duração",
    "artes_e_artesanato": "Não Perecíveis",
    "fraldas_higiene": "Curta Duração",
    "fashion_roupa_infanto_juvenil": "Média Duração",
    "seguros_e_servicos": "Não Perecíveis"
}

# Criação de uma função UDF para mapear as segmentações de validade
def segmentacao_validade(categoria):
    return segmentacoes_validade.get(categoria, "Não Perecíveis")

segmentacao_validade_udf = udf(segmentacao_validade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacao_validade",
    segmentacao_validade_udf(col("product_category_name"))
)


In [7]:
# Definição da segmentação por Utilização Principal
segmentacoes_utilizacao = {
    "beleza_saude": "Uso Diário",
    "informatica_acessorios": "Trabalho",
    "automotivo": "Trabalho",
    "cama_mesa_banho": "Uso Diário",
    "moveis_decoracao": "Lazer",
    "esporte_lazer": "Lazer",
    "perfumaria": "Uso Diário",
    "utilidades_domesticas": "Uso Diário",
    "telefonia": "Trabalho",
    "relogios_presentes": "Presente",
    "alimentos_bebidas": "Uso Diário",
    "bebes": "Uso Diário",
    "papelaria": "Trabalho",
    "tablets_impressao_imagem": "Trabalho",
    "brinquedos": "Lazer",
    "telefonia_fixa": "Trabalho",
    "ferramentas_jardim": "Trabalho",
    "fashion_bolsas_e_acessorios": "Uso Diário",
    "eletroportateis": "Uso Diário",
    "consoles_games": "Lazer",
    "audio": "Lazer",
    "fashion_calcados": "Uso Diário",
    "cool_stuff": "Lazer",
    "malas_acessorios": "Viagem",
    "climatizacao": "Uso Diário",
    "construcao_ferramentas_construcao": "Trabalho",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Uso Diário",
    "construcao_ferramentas_jardim": "Trabalho",
    "fashion_roupa_masculina": "Uso Diário",
    "pet_shop": "Uso Diário",
    "moveis_escritorio": "Trabalho",
    "market_place": "Trabalho",
    "eletronicos": "Trabalho",
    "eletrodomesticos": "Uso Diário",
    "artigos_de_festas": "Lazer",
    "casa_conforto": "Uso Diário",
    "construcao_ferramentas_ferramentas": "Trabalho",
    "agro_industria_e_comercio": "Trabalho",
    "moveis_colchao_e_estofado": "Uso Diário",
    "livros_tecnicos": "Educação",
    "casa_construcao": "Trabalho",
    "instrumentos_musicais": "Lazer",
    "moveis_sala": "Uso Diário",
    "construcao_ferramentas_iluminacao": "Trabalho",
    "industria_comercio_e_negocios": "Trabalho",
    "alimentos": "Uso Diário",
    "artes": "Lazer",
    "moveis_quarto": "Uso Diário",
    "livros_interesse_geral": "Educação",
    "construcao_ferramentas_seguranca": "Trabalho",
    "fashion_underwear_e_moda_praia": "Uso Diário",
    "fashion_esporte": "Lazer",
    "sinalizacao_e_seguranca": "Trabalho",
    "pcs": "Trabalho",
    "artigos_de_natal": "Lazer",
    "fashion_roupa_feminina": "Uso Diário",
    "eletrodomesticos_2": "Uso Diário",
    "livros_importados": "Educação",
    "bebidas": "Uso Diário",
    "cine_foto": "Lazer",
    "la_cuisine": "Uso Diário",
    "musica": "Lazer",
    "casa_conforto_2": "Uso Diário",
    "portateis_casa_forno_e_cafe": "Uso Diário",
    "cds_dvds_musicais": "Lazer",
    "dvds_blu_ray": "Lazer",
    "flores": "Presente",
    "artes_e_artesanato": "Lazer",
    "fraldas_higiene": "Uso Diário",
    "fashion_roupa_infanto_juvenil": "Uso Diário",
    "seguros_e_servicos": "Trabalho"
}

# Criação de uma função UDF para mapear as segmentações de utilização principal
def segmentacao_utilizacao(categoria):
    return segmentacoes_utilizacao.get(categoria, "Uso Diário")

segmentacao_utilizacao_udf = udf(segmentacao_utilizacao, StringType())

# Adição da coluna de segmentação de utilização principal ao DataFrame
products = products.withColumn(
    "segmentacao_utilizacao",
    segmentacao_utilizacao_udf(col("product_category_name"))
)


In [8]:
# Definição da segmentação por Funcionalidade Principal
segmentacoes_funcionalidade = {
    "beleza_saude": "Conforto",
    "informatica_acessorios": "Trabalho",
    "automotivo": "Especializado",
    "cama_mesa_banho": "Essencial",
    "moveis_decoracao": "Conforto",
    "esporte_lazer": "Entretenimento",
    "perfumaria": "Conforto",
    "utilidades_domesticas": "Essencial",
    "telefonia": "Trabalho",
    "relogios_presentes": "Conforto",
    "alimentos_bebidas": "Essencial",
    "bebes": "Essencial",
    "papelaria": "Trabalho",
    "tablets_impressao_imagem": "Trabalho",
    "brinquedos": "Entretenimento",
    "telefonia_fixa": "Trabalho",
    "ferramentas_jardim": "Especializado",
    "fashion_bolsas_e_acessorios": "Conforto",
    "eletroportateis": "Essencial",
    "consoles_games": "Entretenimento",
    "audio": "Entretenimento",
    "fashion_calcados": "Conforto",
    "cool_stuff": "Entretenimento",
    "malas_acessorios": "Conforto",
    "climatizacao": "Essencial",
    "construcao_ferramentas_construcao": "Especializado",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Essencial",
    "construcao_ferramentas_jardim": "Especializado",
    "fashion_roupa_masculina": "Conforto",
    "pet_shop": "Essencial",
    "moveis_escritorio": "Trabalho",
    "market_place": "Trabalho",
    "eletronicos": "Trabalho",
    "eletrodomesticos": "Essencial",
    "artigos_de_festas": "Entretenimento",
    "casa_conforto": "Conforto",
    "construcao_ferramentas_ferramentas": "Especializado",
    "agro_industria_e_comercio": "Especializado",
    "moveis_colchao_e_estofado": "Conforto",
    "livros_tecnicos": "Trabalho",
    "casa_construcao": "Especializado",
    "instrumentos_musicais": "Entretenimento",
    "moveis_sala": "Conforto",
    "construcao_ferramentas_iluminacao": "Especializado",
    "industria_comercio_e_negocios": "Especializado",
    "alimentos": "Essencial",
    "artes": "Entretenimento",
    "moveis_quarto": "Conforto",
    "livros_interesse_geral": "Entretenimento",
    "construcao_ferramentas_seguranca": "Especializado",
    "fashion_underwear_e_moda_praia": "Conforto",
    "fashion_esporte": "Conforto",
    "sinalizacao_e_seguranca": "Especializado",
    "pcs": "Trabalho",
    "artigos_de_natal": "Entretenimento",
    "fashion_roupa_feminina": "Conforto",
    "eletrodomesticos_2": "Essencial",
    "livros_importados": "Entretenimento",
    "bebidas": "Essencial",
    "cine_foto": "Entretenimento",
    "la_cuisine": "Essencial",
    "musica": "Entretenimento",
    "casa_conforto_2": "Conforto",
    "portateis_casa_forno_e_cafe": "Essencial",
    "cds_dvds_musicais": "Entretenimento",
    "dvds_blu_ray": "Entretenimento",
    "flores": "Conforto",
    "artes_e_artesanato": "Entretenimento",
    "fraldas_higiene": "Essencial",
    "fashion_roupa_infanto_juvenil": "Conforto",
    "seguros_e_servicos": "Especializado"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacao_funcionalidade(categoria):
    return segmentacoes_funcionalidade.get(categoria, "Essencial")

# Registro da UDF
segmentacao_funcionalidade_udf = udf(segmentacao_funcionalidade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_funcionalidade",
    segmentacao_funcionalidade_udf(col("product_category_name"))
)

In [9]:
# Definição da segmentação por Nível de Interatividade
segmentacoes_interatividade = {
    "beleza_saude": "Média Interatividade",
    "informatica_acessorios": "Alta Interatividade",
    "automotivo": "Média Interatividade",
    "cama_mesa_banho": "Baixa Interatividade",
    "moveis_decoracao": "Baixa Interatividade",
    "esporte_lazer": "Alta Interatividade",
    "perfumaria": "Média Interatividade",
    "utilidades_domesticas": "Média Interatividade",
    "telefonia": "Alta Interatividade",
    "relogios_presentes": "Baixa Interatividade",
    "alimentos_bebidas": "Baixa Interatividade",
    "bebes": "Média Interatividade",
    "papelaria": "Média Interatividade",
    "tablets_impressao_imagem": "Alta Interatividade",
    "brinquedos": "Alta Interatividade",
    "telefonia_fixa": "Alta Interatividade",
    "ferramentas_jardim": "Média Interatividade",
    "fashion_bolsas_e_acessorios": "Baixa Interatividade",
    "eletroportateis": "Média Interatividade",
    "consoles_games": "Alta Interatividade",
    "audio": "Alta Interatividade",
    "fashion_calcados": "Baixa Interatividade",
    "cool_stuff": "Alta Interatividade",
    "malas_acessorios": "Baixa Interatividade",
    "climatizacao": "Média Interatividade",
    "construcao_ferramentas_construcao": "Média Interatividade",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Baixa Interatividade",
    "construcao_ferramentas_jardim": "Média Interatividade",
    "fashion_roupa_masculina": "Baixa Interatividade",
    "pet_shop": "Média Interatividade",
    "moveis_escritorio": "Baixa Interatividade",
    "market_place": "Média Interatividade",
    "eletronicos": "Alta Interatividade",
    "eletrodomesticos": "Média Interatividade",
    "artigos_de_festas": "Média Interatividade",
    "casa_conforto": "Baixa Interatividade",
    "construcao_ferramentas_ferramentas": "Média Interatividade",
    "agro_industria_e_comercio": "Média Interatividade",
    "moveis_colchao_e_estofado": "Baixa Interatividade",
    "livros_tecnicos": "Média Interatividade",
    "casa_construcao": "Média Interatividade",
    "instrumentos_musicais": "Alta Interatividade",
    "moveis_sala": "Baixa Interatividade",
    "construcao_ferramentas_iluminacao": "Média Interatividade",
    "industria_comercio_e_negocios": "Média Interatividade",
    "alimentos": "Baixa Interatividade",
    "artes": "Alta Interatividade",
    "moveis_quarto": "Baixa Interatividade",
    "livros_interesse_geral": "Baixa Interatividade",
    "construcao_ferramentas_seguranca": "Média Interatividade",
    "fashion_underwear_e_moda_praia": "Baixa Interatividade",
    "fashion_esporte": "Alta Interatividade",
    "sinalizacao_e_seguranca": "Média Interatividade",
    "pcs": "Alta Interatividade",
    "artigos_de_natal": "Média Interatividade",
    "fashion_roupa_feminina": "Baixa Interatividade",
    "eletrodomesticos_2": "Média Interatividade",
    "livros_importados": "Baixa Interatividade",
    "bebidas": "Baixa Interatividade",
    "cine_foto": "Alta Interatividade",
    "la_cuisine": "Média Interatividade",
    "musica": "Alta Interatividade",
    "casa_conforto_2": "Baixa Interatividade",
    "portateis_casa_forno_e_cafe": "Média Interatividade",
    "cds_dvds_musicais": "Alta Interatividade",
    "dvds_blu_ray": "Alta Interatividade",
    "flores": "Baixa Interatividade",
    "artes_e_artesanato": "Alta Interatividade",
    "fraldas_higiene": "Média Interatividade",
    "fashion_roupa_infanto_juvenil": "Baixa Interatividade",
    "seguros_e_servicos": "Média Interatividade"
}
# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacao_interatividade(categoria):
    return segmentacoes_interatividade.get(categoria, "Média Interatividade")

# Registro da UDF
segmentacao_interatividade_udf = udf(segmentacao_interatividade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacao_interatividade",
    segmentacao_interatividade_udf(col("product_category_name"))
)


In [10]:
# Definição da segmentação por Impacto Ambiental
segmentacoes_impacto_ambiental = {
    "beleza_saude": "Médio",
    "informatica_acessorios": "Alto",
    "automotivo": "Alto",
    "cama_mesa_banho": "Baixo",
    "moveis_decoracao": "Médio",
    "esporte_lazer": "Médio",
    "perfumaria": "Médio",
    "utilidades_domesticas": "Médio",
    "telefonia": "Alto",
    "relogios_presentes": "Baixo",
    "alimentos_bebidas": "Baixo",
    "bebes": "Baixo",
    "papelaria": "Baixo",
    "tablets_impressao_imagem": "Alto",
    "brinquedos": "Médio",
    "telefonia_fixa": "Alto",
    "ferramentas_jardim": "Médio",
    "fashion_bolsas_e_acessorios": "Médio",
    "eletroportateis": "Alto",
    "consoles_games": "Alto",
    "audio": "Alto",
    "fashion_calcados": "Médio",
    "cool_stuff": "Alto",
    "malas_acessorios": "Médio",
    "climatizacao": "Alto",
    "construcao_ferramentas_construcao": "Alto",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Baixo",
    "construcao_ferramentas_jardim": "Alto",
    "fashion_roupa_masculina": "Baixo",
    "pet_shop": "Baixo",
    "moveis_escritorio": "Médio",
    "market_place": "Médio",
    "eletronicos": "Alto",
    "eletrodomesticos": "Alto",
    "artigos_de_festas": "Baixo",
    "casa_conforto": "Baixo",
    "construcao_ferramentas_ferramentas": "Alto",
    "agro_industria_e_comercio": "Alto",
    "moveis_colchao_e_estofado": "Médio",
    "livros_tecnicos": "Baixo",
    "casa_construcao": "Alto",
    "instrumentos_musicais": "Médio",
    "moveis_sala": "Médio",
    "construcao_ferramentas_iluminacao": "Alto",
    "industria_comercio_e_negocios": "Alto",
    "alimentos": "Baixo",
    "artes": "Baixo",
    "moveis_quarto": "Médio",
    "livros_interesse_geral": "Baixo",
    "construcao_ferramentas_seguranca": "Alto",
    "fashion_underwear_e_moda_praia": "Baixo",
    "fashion_esporte": "Baixo",
    "sinalizacao_e_seguranca": "Alto",
    "pcs": "Alto",
    "artigos_de_natal": "Baixo",
    "fashion_roupa_feminina": "Baixo",
    "eletrodomesticos_2": "Alto",
    "livros_importados": "Baixo",
    "bebidas": "Baixo",
    "cine_foto": "Alto",
    "la_cuisine": "Baixo",
    "musica": "Baixo",
    "casa_conforto_2": "Baixo",
    "portateis_casa_forno_e_cafe": "Baixo",
    "cds_dvds_musicais": "Baixo",
    "dvds_blu_ray": "Baixo",
    "flores": "Baixo",
    "artes_e_artesanato": "Baixo",
    "fraldas_higiene": "Baixo",
    "fashion_roupa_infanto_juvenil": "Baixo",
    "seguros_e_servicos": "Baixo"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacoe_impacto_ambiental(categoria):
    return segmentacoes_impacto_ambiental.get(categoria, "Média Ambiental")

# Registro da UDF
segmentacoe_impacto_ambiental_udf = udf(segmentacoe_impacto_ambiental, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_impacto_ambiental",
    segmentacoe_impacto_ambiental_udf(col("product_category_name"))
)


In [11]:
# Definição da segmentação por Interação Humana Necessária
segmentacoes_interacao_humana = {
    "beleza_saude": "Manual",
    "informatica_acessorios": "Assistido",
    "automotivo": "Assistido",
    "cama_mesa_banho": "Manual",
    "moveis_decoracao": "Manual",
    "esporte_lazer": "Manual",
    "perfumaria": "Manual",
    "utilidades_domesticas": "Assistido",
    "telefonia": "Assistido",
    "relogios_presentes": "Manual",
    "alimentos_bebidas": "Manual",
    "bebes": "Manual",
    "papelaria": "Manual",
    "tablets_impressao_imagem": "Assistido",
    "brinquedos": "Manual",
    "telefonia_fixa": "Assistido",
    "ferramentas_jardim": "Manual",
    "fashion_bolsas_e_acessorios": "Manual",
    "eletroportateis": "Assistido",
    "consoles_games": "Assistido",
    "audio": "Assistido",
    "fashion_calcados": "Manual",
    "cool_stuff": "Assistido",
    "malas_acessorios": "Manual",
    "climatizacao": "Assistido",
    "construcao_ferramentas_construcao": "Manual",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Manual",
    "construcao_ferramentas_jardim": "Manual",
    "fashion_roupa_masculina": "Manual",
    "pet_shop": "Manual",
    "moveis_escritorio": "Manual",
    "market_place": "Assistido",
    "eletronicos": "Assistido",
    "eletrodomesticos": "Assistido",
    "artigos_de_festas": "Manual",
    "casa_conforto": "Manual",
    "construcao_ferramentas_ferramentas": "Manual",
    "agro_industria_e_comercio": "Assistido",
    "moveis_colchao_e_estofado": "Manual",
    "livros_tecnicos": "Manual",
    "casa_construcao": "Manual",
    "instrumentos_musicais": "Manual",
    "moveis_sala": "Manual",
    "construcao_ferramentas_iluminacao": "Manual",
    "industria_comercio_e_negocios": "Assistido",
    "alimentos": "Manual",
    "artes": "Manual",
    "moveis_quarto": "Manual",
    "livros_interesse_geral": "Manual",
    "construcao_ferramentas_seguranca": "Assistido",
    "fashion_underwear_e_moda_praia": "Manual",
    "fashion_esporte": "Manual",
    "sinalizacao_e_seguranca": "Assistido",
    "pcs": "Assistido",
    "artigos_de_natal": "Manual",
    "fashion_roupa_feminina": "Manual",
    "eletrodomesticos_2": "Assistido",
    "livros_importados": "Manual",
    "bebidas": "Manual",
    "cine_foto": "Assistido",
    "la_cuisine": "Manual",
    "musica": "Manual",
    "casa_conforto_2": "Manual",
    "portateis_casa_forno_e_cafe": "Assistido",
    "cds_dvds_musicais": "Manual",
    "dvds_blu_ray": "Manual",
    "flores": "Manual",
    "artes_e_artesanato": "Manual",
    "fraldas_higiene": "Manual",
    "fashion_roupa_infanto_juvenil": "Manual",
    "seguros_e_servicos": "Assistido"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacoe_interacao_humana(categoria):
    return segmentacoes_interacao_humana.get(categoria, "Interação Humana")

# Registro da UDF
segmentacoe_interacao_humana_udf = udf(segmentacoe_interacao_humana, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_interacao_humana",
    segmentacoe_interacao_humana_udf(col("product_category_name"))
)

In [12]:
# Definição da segmentação por Interação Humana Necessária
segmentacoes_interacao_humana = {
    "beleza_saude": "Manual",
    "informatica_acessorios": "Assistido",
    "automotivo": "Assistido",
    "cama_mesa_banho": "Manual",
    "moveis_decoracao": "Manual",
    "esporte_lazer": "Manual",
    "perfumaria": "Manual",
    "utilidades_domesticas": "Assistido",
    "telefonia": "Assistido",
    "relogios_presentes": "Manual",
    "alimentos_bebidas": "Manual",
    "bebes": "Manual",
    "papelaria": "Manual",
    "tablets_impressao_imagem": "Assistido",
    "brinquedos": "Manual",
    "telefonia_fixa": "Assistido",
    "ferramentas_jardim": "Manual",
    "fashion_bolsas_e_acessorios": "Manual",
    "eletroportateis": "Assistido",
    "consoles_games": "Assistido",
    "audio": "Assistido",
    "fashion_calcados": "Manual",
    "cool_stuff": "Assistido",
    "malas_acessorios": "Manual",
    "climatizacao": "Assistido",
    "construcao_ferramentas_construcao": "Manual",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Manual",
    "construcao_ferramentas_jardim": "Manual",
    "fashion_roupa_masculina": "Manual",
    "pet_shop": "Manual",
    "moveis_escritorio": "Manual",
    "market_place": "Assistido",
    "eletronicos": "Assistido",
    "eletrodomesticos": "Assistido",
    "artigos_de_festas": "Manual",
    "casa_conforto": "Manual",
    "construcao_ferramentas_ferramentas": "Manual",
    "agro_industria_e_comercio": "Assistido",
    "moveis_colchao_e_estofado": "Manual",
    "livros_tecnicos": "Manual",
    "casa_construcao": "Manual",
    "instrumentos_musicais": "Manual",
    "moveis_sala": "Manual",
    "construcao_ferramentas_iluminacao": "Manual",
    "industria_comercio_e_negocios": "Assistido",
    "alimentos": "Manual",
    "artes": "Manual",
    "moveis_quarto": "Manual",
    "livros_interesse_geral": "Manual",
    "construcao_ferramentas_seguranca": "Assistido",
    "fashion_underwear_e_moda_praia": "Manual",
    "fashion_esporte": "Manual",
    "sinalizacao_e_seguranca": "Assistido",
    "pcs": "Assistido",
    "artigos_de_natal": "Manual",
    "fashion_roupa_feminina": "Manual",
    "eletrodomesticos_2": "Assistido",
    "livros_importados": "Manual",
    "bebidas": "Manual",
    "cine_foto": "Assistido",
    "la_cuisine": "Manual",
    "musica": "Manual",
    "casa_conforto_2": "Manual",
    "portateis_casa_forno_e_cafe": "Assistido",
    "cds_dvds_musicais": "Manual",
    "dvds_blu_ray": "Manual",
    "flores": "Manual",
    "artes_e_artesanato": "Manual",
    "fraldas_higiene": "Manual",
    "fashion_roupa_infanto_juvenil": "Manual",
    "seguros_e_servicos": "Assistido"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacoe_interacao_humana(categoria):
    return segmentacoes_interacao_humana.get(categoria, "Interação Humana")

# Registro da UDF
segmentacoe_interacao_humana_udf = udf(segmentacoe_interacao_humana, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_interacao_humana",
    segmentacoe_interacao_humana_udf(col("product_category_name"))
)

In [13]:
import pandas as pd
from google.colab import files

# Supondo que você já tenha um DataFrame Spark chamado product_category_name_translation
# Converter o DataFrame Spark para um DataFrame Pandas para salvar como CSV
df_pandas = products.toPandas()

# Especificar o caminho para salvar o arquivo CSV
caminho_arquivo = '/content/products.csv'

# Salvar o DataFrame Pandas como CSV
df_pandas.to_csv(caminho_arquivo, index=False)

# Fazer o download do arquivo CSV
files.download(caminho_arquivo)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>